### Load 10k fillings

In [3]:
#Install datasets
!pip install datasets

In [4]:
# Import 10k sec fillings from hugging face dataset
import datasets
raw_dataset = datasets.load_dataset("JanosAudran/financial-reports-sec", "small_full")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset financial_reports_sec downloaded and prepared to /root/.cache/huggingface/datasets/JanosAudran___financial_reports_sec/small_full/1.1.1/e9328f71b08580052f4ba03a561977d78e8024c5257d4a71bb465988b4a71b47. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# Import pandas to work with dataframes
import pandas as pd

In [6]:
# Convert the data stored with a Dataframe dict into individual dataframes
df_1= raw_dataset["train"].to_pandas()
df_2=raw_dataset["validation"].to_pandas()
df_3=raw_dataset["test"].to_pandas()

# Concat to clean and preprocess all the dataframes in one step
concatenated_df = pd.concat([df_1, df_2, df_3], ignore_index=True)

In [7]:
# The current dataframe
concatenated_df.head()

,cik,sentence,section,labels,filingDate,name,docID,sentenceID,sentenceCount,tickers,exchanges,entityType,sic,stateOfIncorporation,tickerCount,acceptanceDateTime,form,reportDate,returns
0,0000001750,ITEM 1.BUSINESS General AAR CORP. and its subs...,0,"{'1d': 0, '5d': 1, '30d': 0}",2020-07-21,AAR CORP,0000001750_10-K_2020,0000001750_10-K_2020_section_1_0,1,[AIR],[NYSE],operating,3720,DE,1,2020-07-21T17:19:15.000Z,10-K,2020-05-31,{'1d': {'closePriceEndDate': 19.01000022888183...
1,0000001750,"AAR was founded in 1951, organized in 1955 and...",0,"{'1d': 0, '5d': 1, '30d': 0}",2020-07-21,AAR CORP,0000001750_10-K_2020,0000001750_10-K_2020_section_1_1,2,[AIR],[NYSE],operating,3720,DE,1,2020-07-21T17:19:15.000Z,10-K,2020-05-31,{'1d': {'closePriceEndDate': 19.01000022888183...
2,0000001750,We are a diversified provider of products and ...,0,"{'1d': 0, '5d': 1, '30d': 0}",2020-07-21,AAR CORP,0000001750_10-K_2020,0000001750_10-K_2020_section_1_2,3,[AIR],[NYSE],operating,3720,DE,1,2020-07-21T17:19:15.000Z,10-K,2020-05-31,{'1d': {'closePriceEndDate': 19.01000022888183...
3,0000001750,Fiscal 2020 began with strategic initiatives f...,0,"{'1d': 0, '5d': 1, '30d': 0}",2020-07-21,AAR CORP,0000001750_10-K_2020,0000001750_10-K_2020_section_1_3,4,[AIR],[NYSE],operating,3720,DE,1,2020-07-21T17:19:15.000Z,10-K,2020-05-31,{'1d': {'closePriceEndDate': 19.01000022888183...
4,0000001750,Our momentum from a successful fiscal 2019 car...,0,"{'1d': 0, '5d': 1, '30d': 0}",2020-07-21,AAR CORP,0000001750_10-K_2020,0000001750_10-K_2020_section_1_4,5,[AIR],[NYSE],operating,3720,DE,1,2020-07-21T17:19:15.000Z,10-K,2020-05-31,{'1d': {'closePriceEndDate': 19.01000022888183...


In [8]:
# Sepearting out the returns dict for only 1d to be able to understand immediate market chnages due to SEC fillings
concatenated_df['1D_return']=concatenated_df['returns'].apply(lambda x: x.get('1d', None))

# get sentimental labels on a per filling bases from the market reaction for 1 day after the filling date
positive_threshold = 0.02
negative_threshold = -0.02

def assign_sentiment_label(row):
  if row['1D_return']['ret'] > positive_threshold:
    return 'positive'
  elif row['1D_return']['ret'] < negative_threshold:
    return 'negative'
  else:
    return 'neutral'

concatenated_df['sentiment']=concatenated_df.apply(assign_sentiment_label, axis=1)



In [9]:
# Filter the dataframe to only include section 1A, section 7, and 7A of the reports as they are known to be most correlated to stock prices
filtered_df = concatenated_df[concatenated_df['section'].isin([1,8,9])].reset_index(drop=True)

# Filter the dataframe to only include reports from 2016 onwards (2016 - 2020)
filtered_df['filingDate'] = pd.to_datetime(filtered_df['filingDate'])
filtered_df = filtered_df[filtered_df['filingDate'].dt.year >= 2016]

In [10]:
# Group all sentences from a section into one string 
grouped_df = filtered_df.groupby(['docID', 'section'], as_index=False).agg({
    'cik': 'first',
    'sentence': ' '.join,
    'filingDate': 'first',
    'name': 'first',
    'tickers': 'first',
    'exchanges': 'first',
    'entityType': 'first',
    'sic': 'first',
    'stateOfIncorporation': 'first',
    'acceptanceDateTime': 'first',
    'reportDate': 'first',
    'sentiment': 'first'})

In [11]:
# The final df we will be using for sentiment analysis
grouped_df

,docID,section,cik,sentence,filingDate,name,tickers,exchanges,entityType,sic,stateOfIncorporation,acceptanceDateTime,reportDate,sentiment
0,0000001750_10-K_2016,1,0000001750,ITEM 1A. RISK FACTORS The following is a descr...,2016-07-13,AAR CORP,[AIR],[NYSE],operating,3720,DE,2016-07-13T15:13:34.000Z,2016-05-31,negative
1,0000001750_10-K_2016,8,0000001750,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,2016-07-13,AAR CORP,[AIR],[NYSE],operating,3720,DE,2016-07-13T15:13:34.000Z,2016-05-31,negative
2,0000001750_10-K_2016,9,0000001750,ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSU...,2016-07-13,AAR CORP,[AIR],[NYSE],operating,3720,DE,2016-07-13T15:13:34.000Z,2016-05-31,negative
3,0000001750_10-K_2017,1,0000001750,ITEM 1A. RISK FACTORS The following is a descr...,2017-07-12,AAR CORP,[AIR],[NYSE],operating,3720,DE,2017-07-12T15:26:07.000Z,2017-05-31,positive
4,0000001750_10-K_2017,8,0000001750,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,2017-07-12,AAR CORP,[AIR],[NYSE],operating,3720,DE,2017-07-12T15:26:07.000Z,2017-05-31,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,0001699150_10-K_2019,8,0001699150,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...,2020-02-26,Ingersoll Rand Inc.,[IR],[NYSE],operating,3560,DE,2020-02-26T16:04:02.000Z,2019-12-31,negative
222,0001699150_10-K_2019,9,0001699150,ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSU...,2020-02-26,Ingersoll Rand Inc.,[IR],[NYSE],operating,3560,DE,2020-02-26T16:04:02.000Z,2019-12-31,negative
223,0001699150_10-K_2020,1,0001699150,ITEM 1A. RISK FACTORS The following risk facto...,2021-02-26,Ingersoll Rand Inc.,[IR],[NYSE],operating,3560,DE,2021-02-26T16:03:05.000Z,2020-12-31,positive
224,0001699150_10-K_2020,8,0001699150,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...,2021-02-26,Ingersoll Rand Inc.,[IR],[NYSE],operating,3560,DE,2021-02-26T16:03:05.000Z,2020-12-31,positive


In [12]:
import string

# Preprocess the test to lowercase and remove punctuation
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text

grouped_df['sentence'] =grouped_df['sentence'].apply(preprocess_text)

# Map sentiment analysis
sentiment_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
grouped_df['sentiment_label'] = grouped_df['sentiment'].map(sentiment_mapping)



In [13]:
# Split data into train and test split
from sklearn.model_selection import train_test_split
X = grouped_df.drop(columns=['sentiment', 'sentiment_label'])
y = grouped_df['sentiment_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state=42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Training a neural model for sentiment analysis

In [64]:
# All necessary imports
import torch
import torch.nn as nn
from transformers import BertTokenizerFast, AutoModel, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification
from sklearn.model_selection import KFold
import string
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt 

In [13]:
# Load FinBERT tokenizer which is used to enocde and decode the text
tokenizer = BertTokenizerFast.from_pretrained('yiyanghkust/finbert-fls')

# Load finbert pretrained model
finbert = AutoModel.from_pretrained('yiyanghkust/finbert-fls')

Some weights of the model checkpoint at yiyanghkust/finbert-fls were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# Convert texts and labels into lists to pass into pytorch network
texts = X_train['sentence'].tolist()
labels = y_train.tolist()
sections = X_train['section'].tolist()
states = X_train['stateOfIncorporation'].tolist()
times = X_train['filingDate'].tolist()

### Running baseline FinBERT on validation data

In [ ]:
# Using FinBert as is to predict sentiment on validaiton data
# Define k-fold cross-validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

fold_accuracies = []

for fold, (train_indices, val_indices) in enumerate(kf.split(texts)):
    print(f"Fold {fold + 1}/{num_folds}")

    train_texts = [texts[i] for i in train_indices]
    train_labels = [labels[i] for i in train_indices]
    val_texts = [texts[i] for i in val_indices]
    val_labels = [labels[i] for i in val_indices]

    # Tokenize and encode the training data
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
    train_labels = torch.tensor(train_labels)

    # Tokenize and encode the validation data
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
    val_labels = torch.tensor(val_labels)

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(
        train_encodings['input_ids'].clone().detach(),
        train_encodings['attention_mask'].clone().detach(),
        train_labels.clone().detach()
    )
    val_dataset = torch.utils.data.TensorDataset(
        val_encodings['input_ids'].clone().detach(),
        val_encodings['attention_mask'].clone().detach(),
        val_labels.clone().detach()
    )

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

    val_predictions = []
    for batch in val_loader:
        inputs, masks, targets = batch
        with torch.no_grad():
            # Instantiate BERT model
            model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)
            model.to(device)
            model.eval()  # Set the model to evaluation mode

            inputs, masks, targets = inputs.to(device), masks.to(device), targets.to(device)

            outputs = model(input_ids=inputs, attention_mask=masks)
            val_predictions.extend(torch.argmax(outputs.logits, dim=1).tolist())

    correct_predictions = sum([1 if pred == target else 0 for pred, target in zip(val_predictions, val_labels)])
    fold_accuracy = correct_predictions / len(val_labels)
    fold_accuracies.append(fold_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(fold_accuracies) / len(fold_accuracies)
print(f"Avg Validation Accuracy: {overall_accuracy}")
print("=" * 40)

### Running FinBERT on training data

In [ ]:
# Running regular FinBERT model, but using loss function, optimizer, and training epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with torch.cuda.device('cuda:0'):
    # Load model with specified architecture
    model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)

    # Move model to GPU
    model.to(device)

    # Define loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

    # Define k-fold cross-validation
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    fold_accuracies = []
    fold_val_loss = []


    for fold, (train_indices, val_indices) in enumerate(kf.split(texts)):
        print(f"Fold {fold + 1}/{num_folds}")

        train_texts = [texts[i] for i in train_indices]
        train_labels = [labels[i] for i in train_indices]
        val_texts = [texts[i] for i in val_indices]
        val_labels = [labels[i] for i in val_indices]

        # Tokenize and encode the training data
        train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
        train_labels = torch.tensor(train_labels)

        # Tokenize and encode the validation data
        val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
        val_labels = torch.tensor(val_labels)


        # Create data loaders
        train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'].clone().detach(), 
        train_encodings['attention_mask'].clone().detach(), 
        train_labels.clone().detach())
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

        val_dataset = torch.utils.data.TensorDataset(val_encodings['input_ids'].clone().detach(),
        val_encodings['attention_mask'].clone().detach(),
        val_labels.clone().detach())
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)


        training_loss_all = 0.0
        num_epochs = 5


        # Train the model
        for epoch in range(num_epochs):
            model.train()  # Set the model to training model
            train_loss = 0.0
            for batch in train_loader:
                optimizer.zero_grad()
                inputs, masks, targets = batch
                inputs, masks, targets = inputs.to(device), masks.to(device), targets.to(device)

                outputs = model(input_ids=inputs, attention_mask=masks, labels=targets)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                
                # Clear batch-related tensors from memory
                del inputs, masks, targets, outputs
                torch.cuda.empty_cache()

            epoch_training_loss = train_loss/len(train_loader)
            training_loss_all += epoch_training_loss
            print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {epoch_training_loss}")

        # Evaluate on validation set
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            val_loss = 0.0
            val_predictions = []
            for batch in val_loader:
                inputs, masks,targets = batch
                inputs, masks, targets = inputs.to(device), masks.to(device), targets.to(device)

                outputs = model(input_ids=inputs, attention_mask=masks, labels=targets)
                loss = outputs.loss
                val_loss += loss.item()
                val_predictions.extend(torch.argmax(outputs.logits, dim=1).tolist())
                
                # Clear batch-related tensors from memory
                del inputs, masks, targets, outputs
                torch.cuda.empty_cache()

            fold_validation_loss = val_loss / len(val_loader)
            correct_predictions = sum([1 if pred == target else 0 for pred, target in zip(val_predictions, val_labels)])
            fold_accuracy = correct_predictions / len(val_labels)

            fold_accuracies.append(fold_accuracy)
            fold_val_loss.append(fold_validation_loss)
            torch.cuda.empty_cache()
            
        # Clear fold-related tensors from memory
        del train_encodings, train_labels, val_encodings, val_labels, train_loader, val_loader
        torch.cuda.empty_cache()

    # Calculate overall accuracy and validation loss
    overall_accuracy = sum(fold_accuracies) / len(fold_accuracies)
    overall_validation_loss = sum(fold_val_loss) / len(fold_val_loss)
    training_loss = training_loss_all / (num_epochs)

    print(f"Avg Validation Loss: {overall_validation_loss}")
    print(f"Avg Training Loss: {training_loss}")
    print(f"Avg Validation Accuracy: {overall_accuracy}")
    print("="*40)

### Creating a sentiment model using FinBERT

In [15]:
# Freeze finbert parameters - would be helpful if we had a large dataset and didn't want to change the weights of finBERT
#for param in finbert.parameters():
    #param.requires_grad = False

In [24]:
# Neural network built of pre-trained FinBERT
class CustomSentimentModel(nn.Module):
    def __init__(self, finbert, hidden_size, num_classes = 3, dropout = 0.1):
        super(CustomSentimentModel, self).__init__()
        self.finbert = finbert
        # dropout layer
        self.dropout = nn.Dropout(dropout)

        # relu activation function
        self.relu =  nn.ReLU()

        # Batch normalization layers
        self.bn1 = nn.BatchNorm1d(768 + 3)
        self.bn2 = nn.BatchNorm1d(hidden_size)


        # dense layer 1
        self.fc1 = nn.Linear(768 + 3,hidden_size)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(hidden_size,num_classes)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids, attention_mask, features):
        pooled_output = self.finbert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        combined_features = torch.cat((pooled_output, features), dim=1)
        x = self.bn1(combined_features)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.bn2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [17]:
# The hyperparameters to tune
epochs_values = [5, 10]
batch_size_values = [16,32]
hidden_size_values = [256, 512]
dropout_prob_values = [0.1, 0.2]

In [47]:
# Store the results
results = []

In [52]:
# Iterate over hyperparameter combinations keeping track of loss and accuracy to find the best model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with torch.cuda.device('cuda:0'):
    for num_epochs in epochs_values:
        for batch_size in batch_size_values:
            for hidden_size in hidden_size_values:
                for dropout_prob in dropout_prob_values:
                    print(f"Ablation Study on training data: epochs={num_epochs}, batch_size={batch_size},  "
                    f"hidden_size={hidden_size}, dropout_prob={dropout_prob}")

                    # Load model with specified architecture
                    custom_model = CustomSentimentModel(finbert, hidden_size=hidden_size, num_classes=3, dropout=dropout_prob)

                    # Move model to GPU
                    custom_model.to(device)

                    # Define loss function and optimizer
                    loss_fn = nn.CrossEntropyLoss()
                    optimizer = torch.optim.AdamW(custom_model.parameters(), lr=1e-3, weight_decay=1e-2)
                    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

                    # Define k-fold cross-validation
                    num_folds = 5
                    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

                    fold_accuracies = []
                    fold_val_loss = []
                    
                    # Outside of the loop for hyperparameter combinations
                    learning_curve_data = {'training_loss': [], 'validation_loss': []}


                    for fold, (train_indices, val_indices) in enumerate(kf.split(texts)):
                        print(f"Fold {fold + 1}/{num_folds}")

                        train_texts = [texts[i] for i in train_indices]
                        train_labels = [labels[i] for i in train_indices]
                        val_texts = [texts[i] for i in val_indices]
                        val_labels = [labels[i] for i in val_indices]
                        train_sections = [sections[i] for i in train_indices]
                        train_states = [states[i] for i in train_indices]
                        train_times = [times[i] for i in train_indices]
                        val_sections = [sections[i] for i in val_indices]
                        val_states = [states[i] for i in val_indices]
                        val_times = [times[i] for i in val_indices]

                        # Tokenize and encode the training data
                        train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
                        train_labels = torch.tensor(train_labels)

                        # Other features (section, state, time) need to be encoded and concatenated
                        train_time_features = torch.tensor([timestamp.timestamp() for timestamp in train_times]).unsqueeze(1)
                        label_encoder_state = LabelEncoder()
                        label_encoder_section = LabelEncoder()
                        train_state_labels = label_encoder_state.fit_transform(train_states)
                        train_section_labels = label_encoder_section.fit_transform(train_sections)
                        
                        train_state_features = torch.tensor(train_state_labels, dtype=torch.float32).unsqueeze(1)
                        train_section_features = torch.tensor(train_section_labels, dtype=torch.float32).unsqueeze(1)

                        # Combine all features horizontally
                        train_additional_features = torch.cat((train_section_features, train_state_features, train_time_features), dim=1)

                        # Tokenize and encode the validation data
                        val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
                        val_labels = torch.tensor(val_labels)


                        # Other features (section, state, time) need to be encoded and concatenated
                        val_time_features = torch.tensor([timestamp.timestamp() for timestamp in val_times]).unsqueeze(1)

                        label_encoder_state_val = LabelEncoder()
                        label_encoder_section_val = LabelEncoder()
                        val_state_labels = label_encoder_state_val.fit_transform(val_states)
                        val_section_labels = label_encoder_section_val.fit_transform(val_sections)
                        
                        
                        val_state_features = torch.tensor(val_state_labels, dtype=torch.float32).unsqueeze(1)
                        val_section_features = torch.tensor(val_section_labels, dtype=torch.float32).unsqueeze(1)

                        # Combine all features horizontally
                        val_additional_features = torch.cat((val_section_features, val_state_features, val_time_features), dim=1)


                        # Create data loaders
                        train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'].clone().detach(), 
                                                              train_encodings['attention_mask'].clone().detach(),
                                                              train_additional_features, 
                                                              train_labels.clone().detach())
                        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

                        val_dataset = torch.utils.data.TensorDataset(val_encodings['input_ids'].clone().detach(),
                                                            val_encodings['attention_mask'].clone().detach(),
                                                            val_additional_features,
                                                            val_labels.clone().detach())
                        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


                        training_loss_all = 0.0


                        # Train the model
                        for epoch in range(num_epochs):
                            custom_model.train()  # Set the model to training model
                            train_loss = 0.0
                            for batch in train_loader:
                                optimizer.zero_grad()
                                inputs, masks, features, targets = batch
                                inputs, masks, features, targets = inputs.to(device), masks.to(device), features.to(device), targets.to(device)

                                outputs = custom_model(input_ids=inputs, attention_mask=masks, features=features)
                                loss = loss_fn(outputs, targets)
                                loss.backward()

                                # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
                                torch.nn.utils.clip_grad_norm_(custom_model.parameters(), 1.0)

                                optimizer.step()
                                scheduler.step()
                                train_loss += loss.item()

                            epoch_training_loss = train_loss/len(train_loader)
                            training_loss_all += epoch_training_loss
                            print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {epoch_training_loss}")
                            learning_curve_data['training_loss'].append(epoch_training_loss)

                        # Evaluate on validation set
                        custom_model.eval()  # Set the model to evaluation mode
                        with torch.no_grad():
                            val_loss = 0.0
                            val_predictions = []
                            for batch in val_loader:
                                inputs, masks, features, targets = batch
                                inputs, masks, features, targets = inputs.to(device), masks.to(device), features.to(device), targets.to(device)

                                outputs = custom_model(input_ids=inputs, attention_mask=masks, features=features)
                                loss = loss_fn(outputs, targets)
                                val_loss += loss.item()
                                val_predictions.extend(torch.argmax(outputs, dim=1).tolist())

                            fold_validation_loss = val_loss / len(val_loader)
                            correct_predictions = sum([1 if pred == target else 0 for pred, target in zip(val_predictions, val_labels)])
                            fold_accuracy = correct_predictions / len(val_labels)

                            fold_accuracies.append(fold_accuracy)
                            fold_val_loss.append(fold_validation_loss)
                            learning_curve_data['validation_loss'].append(fold_validation_loss)


                    # Calculate overall accuracy and validation loss
                    overall_accuracy = sum(fold_accuracies) / len(fold_accuracies)
                    overall_validation_loss = sum(fold_val_loss) / len(fold_val_loss)
                    training_loss = training_loss_all / (num_epochs)

                    results.append({
                    'epochs': num_epochs,
                    'batch_size': batch_size,
                    'hidden_size': hidden_size,
                    'dropout_prob': dropout_prob,
                    'avg_validation_loss': overall_validation_loss,
                    'avg_training_loss': training_loss,
                    'avg_validation_accuracy': overall_accuracy
                    })

                    print(f"Avg Validation Loss: {overall_validation_loss}")
                    print(f"Avg Training Loss: {training_loss}")
                    print(f"Avg Validation Accuracy: {overall_accuracy}")
                    print("="*40)
    torch.cuda.empty_cache()

Ablation Study on training data: epochs=5, batch_size=16,  hidden_size=256, dropout_prob=0.1
Fold 1/5
Epoch 1/5 - Training Loss: 1.1544514629575942
Epoch 2/5 - Training Loss: 1.1503188212712605
Epoch 3/5 - Training Loss: 1.1618451277414958
Epoch 4/5 - Training Loss: 1.1903991036944919
Epoch 5/5 - Training Loss: 1.123514731725057
Fold 2/5
Epoch 1/5 - Training Loss: 1.1409244537353516
Epoch 2/5 - Training Loss: 1.1939329703648884
Epoch 3/5 - Training Loss: 1.1398513184653387
Epoch 4/5 - Training Loss: 1.1667167610592313
Epoch 5/5 - Training Loss: 1.148673004574246
Fold 3/5
Epoch 1/5 - Training Loss: 1.13162757953008
Epoch 2/5 - Training Loss: 1.1601971785227458
Epoch 3/5 - Training Loss: 1.1562539206610785
Epoch 4/5 - Training Loss: 1.1701712475882635
Epoch 5/5 - Training Loss: 1.208330856429206
Fold 4/5
Epoch 1/5 - Training Loss: 1.1369435323609247
Epoch 2/5 - Training Loss: 1.1125637425316706
Epoch 3/5 - Training Loss: 1.1325271129608154
Epoch 4/5 - Training Loss: 1.1419297721650865
Ep

From these results it's evident that the best model is one that is trained on epochs=5, batch_size=32,  hidden_size=256, dropout_prob=0.2. This is the one with the highest validation acuracy at 48% and a Avg Validation Loss: 1.0891265551249187
Avg Training Loss: 1.1690920220481025. 

### Running the best trained model on test data 

In [17]:
test_text = X_test['sentence'].tolist()
test_label= = y_test.tolist()
test_sections = X_test['section'].tolist()
test_state = X_test['stateOfIncorporation'].tolist()
test_time = X_test['filingDate'].tolist()

In [ ]:
# Imports to display misclassified text
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set the model to evaluation mode
final_model = CustomSentimentModel(finbert, hidden_size=256, num_classes=3, dropout=0.2)
final_model.to(device)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(final_model.parameters(), lr=1e-3, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)


# Tokenize and encode the training data
train_encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
train_labels = torch.tensor(labels)

# Other features (section, state, time) need to be encoded and concatenated
train_time_features = torch.tensor([timestamp.timestamp() for timestamp in times]).unsqueeze(1)
label_encoder_state = LabelEncoder()
label_encoder_section = LabelEncoder()
train_state_labels = label_encoder_state.fit_transform(states)
train_section_labels = label_encoder_section.fit_transform(sections)

train_state_features = torch.tensor(train_state_labels, dtype=torch.float32).unsqueeze(1)
train_section_features = torch.tensor(train_section_labels, dtype=torch.float32).unsqueeze(1)



# Combine all features horizontally
train_additional_features = torch.cat((train_section_features, train_state_features, train_time_features), dim=1)



# Tokenize and encode the test data
test_encodings = tokenizer(test_text, truncation=True, padding=True, return_tensors='pt', max_length=512)
test_labels = torch.tensor(test_label)

# Other features (section, state, time) need to be encoded and concatenated
test_time_features = torch.tensor([timestamp.timestamp() for timestamp in test_times]).unsqueeze(1)
label_encoder_state_test = LabelEncoder()
label_encoder_section_test = LabelEncoder()
test_state_labels = label_encoder_state_test.fit_transform(test_state)
test_section_labels = label_encoder_section_test.fit_transform(test_sections)

test_state_features = torch.tensor(test_state_labels, dtype=torch.float32).unsqueeze(1)
test_section_features = torch.tensor(test_section_labels, dtype=torch.float32).unsqueeze(1)


# Combine all features horizontally
test_additional_features = torch.cat((test_section_features, test_state_features, test_time_features), dim=1)


# Create data loader
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'].clone().detach(), 
                                                              train_encodings['attention_mask'].clone().detach(),
                                                              train_additional_features, 
                                                              train_labels.clone().detach())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'].clone().detach(), 
                                                              test_encodings['attention_mask'].clone().detach(),
                                                              test_additional_features, 
                                                              test_labels.clone().detach())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

num_epochs = 5
for epoch in range(num_epochs):
    final_model.train()  # Set the model to training model
    train_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs, masks, features, targets = batch
        inputs, masks, features, targets = inputs.to(device), masks.to(device), features.to(device), targets.to(device)

        outputs = final_model(input_ids=inputs, attention_mask=masks, features=features)
        loss = loss_fn(outputs, targets)
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(final_model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

final_model.eval()
# Evaluate on test set
with torch.no_grad():
    test_predictions = []
    for batch in test_loader:
        inputs, masks, features, targets = batch
        inputs, masks, features, targets = inputs.to(device), masks.to(device), features.to(device), targets.to(device)

        outputs = final_model(input_ids=inputs, attention_mask=masks, features=features)
        test_predictions.extend(torch.argmax(outputs, dim=1).tolist())
        
test_accuracy = accuracy_score(test_labels.tolist(), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}") 

# Generate classification report
target_names = ['class 0', 'class 1', 'class 2'] 
classification_rep = classification_report(test_labels.tolist(), test_predictions, target_names=target_names)
print(classification_rep)

# Analyze Misclassified items:
for text, true_label, predicted_label in zip(test_text, test_labels.tolist(), test_predictions):
    if true_label != predicted_label:
        print("Misclassified Item:")
        print("True Label:", true_label)
        print("Predicted Label:", predicted_label)
        print("="*40)
        
        # Generate word cloud
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
        
        # Display word cloud
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.show()
        
        print("="*40)


### Running the test data on the general FinBERT model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)
base_model.to(device)
base_model.eval()


# Tokenize and encode the training data
test_encodings = tokenizer(test_text, truncation=True, padding=True, return_tensors='pt', max_length=512)
test_labels = torch.tensor(test_labels)

# Create data loaders
test_dataset = torch.utils.data.TensorDataset(
    test_encodings['input_ids'].clone().detach(),
    test_encodings['attention_mask'].clone().detach(),
    test_labels.clone().detach())
    
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)

# Evaluate on test set
with torch.no_grad():
    test_predictions = []
    misclassified_indices = []
    for batch in test_loader:
        inputs, masks, targets = batch
        inputs, masks, targets = inputs.to(device), masks.to(device), targets.to(device)

        outputs = base_model(input_ids=inputs, attention_mask=masks)
        test_predictions.extend(torch.argmax(outputs.logits, dim=1).tolist())

test_accuracy = accuracy_score(test_labels.tolist(), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate classification report
target_names = ['class 0', 'class 1', 'class 2']  # Replace with your class names
classification_rep = classification_report(test_labels.tolist(), test_predictions, target_names=target_names)
print(classification_rep)